In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn import utils
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # fancy statistics plots
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import geopandas as gpd
import scipy
from scipy.optimize import curve_fit
import datetime
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import seaborn as sns
from plotly.subplots import make_subplots
%matplotlib inline


import plotly.tools as tls
import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv('../input/covid19-in-turkey/covid_19_data_tr.csv')
dataset

In [ ]:
#Veri Setindeki Anahtarlar
print(dataset.keys())

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

In [ ]:
#Verinin Yapısı
print(dataset.shape)

In [ ]:
#Vaka Sayılarının Ortalamasını Görüntüleme
dataset.groupby('Country/Region')['Confirmed'].apply(lambda x: np.mean(x))

In [ ]:
cases = dataset.groupby(['Last_Update']).sum().reset_index()


In [ ]:
array = dataset.values
X = array[:,0:5]
y = array[:,0:1]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

print("Dataframe boyutu: ",dataset.shape)
print("Eğitim verisi boyutu: ",X_train.shape, Y_train.shape)
print("Test verisi boyutu: ",X_validation.shape, Y_validation.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(cases["Last_Update"],
        cases["Confirmed"],
        color="y");
ax.set_title("Turkey Confirmed Cases Per Day");
ax.spines["top"].set_visible(False);
ax.spines["right"].set_visible(False);

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(cases["Last_Update"],
        cases["Deaths"],
        color="r");
ax.set_title("Turkey Confirmed Deaths Per Day");
ax.spines["top"].set_visible(False);
ax.spines["right"].set_visible(False);

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(cases["Last_Update"],
        cases["Recovered"],
        color="b");
ax.set_title("Turkey Recovered Cases Per Day");
ax.spines["top"].set_visible(False);
ax.spines["right"].set_visible(False);

In [ ]:
confirmed_fig = px.line(x=dataset.index,y=dataset['Confirmed'], color=px.Constant("Confirmed"),
             labels=dict(x="Last_Update", y="# of  Confirmed", color="Time Period"),
            title = 'Daily Confirmed Trend').update_layout(plot_bgcolor='aqua')
confirmed_fig.show()

In [ ]:
Deaths_fig = px.line(x=dataset.index,y=dataset['Deaths'], color=px.Constant("Deaths"),
             labels=dict(x="Last_Update", y="# of  Deaths", color="Time Period"),
            title = 'Daily Deaths Trend').update_layout(plot_bgcolor='aqua')
Deaths_fig.show()

In [ ]:
Recovered_fig = px.line(x=dataset.index,y=dataset['Recovered'], color=px.Constant("Recovered"),
             labels=dict(x="Last_Update", y="# of  Recovered", color="Time Period"),
            title = 'Daily Recovered Trend').update_layout(plot_bgcolor='aqua')
Recovered_fig.show()

In [ ]:
#Korelasyon Gösterimi
import seaborn as sns
corr = dataset.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values) #Korelasyon gösterimi

In [ ]:
dataset.hist()
pyplot.show()

In [ ]:
dataset['Active']=dataset['Confirmed']-dataset['Deaths']-dataset['Recovered']
temp = dataset.groupby('Last_Update')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Last_Update", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')


fig = px.area(temp, x="Last_Update", y="Count", color='Case',
             title='Yayılma Hızı', color_discrete_sequence = ['#21bf73', '#ff2e63', '#fe9801'])
fig.show()

In [ ]:
dataset.plot(x='Last_Update', y='Confirmed', style='*',color="b") 

In [ ]:
dataset.plot(x='Last_Update', y='Deaths', style='*',color="r") 

In [ ]:
dataset.plot(x='Last_Update', y='Recovered', style='*',color="g") 

In [ ]:
dataset.plot(kind='box', subplots=True, layout=(3,3), sharex=False, sharey=False)
pyplot.show()

In [ ]:
#İl Bazında Vaka Sayılarını Gösteren Histogram Grafiği
import pandas as pd
import seaborn as sns
%matplotlib inline
dataset=pd.read_csv("../input/covid19-in-turkey/covid_19_data_tr.csv")
plt.figure(figsize = (40,90))
groupedvalues=dataset.groupby('Last_Update').sum().reset_index()
g=sns.barplot(x='Confirmed', y='Last_Update',data=groupedvalues)
plt.title('Türkiye Günlük Vaka Sayıları');


In [ ]:
#İl Bazında Vaka Sayılarını Gösteren Histogram Grafiği
import pandas as pd
import seaborn as sns
%matplotlib inline
dataset=pd.read_csv("../input/covid19-in-turkey/covid_19_data_tr.csv")
plt.figure(figsize = (40,90))
groupedvalues=dataset.groupby('Last_Update').sum().reset_index()
g=sns.barplot(x='Deaths', y='Last_Update',data=groupedvalues)
plt.title('Türkiye Günlük Ölüm Sayıları');


In [ ]:
#İl Bazında Vaka Sayılarını Gösteren Histogram Grafiği
import pandas as pd
import seaborn as sns
%matplotlib inline
dataset=pd.read_csv("../input/covid19-in-turkey/covid_19_data_tr.csv")
plt.figure(figsize = (40,90))
groupedvalues=dataset.groupby('Last_Update').sum().reset_index()
g=sns.barplot(x='Recovered', y='Last_Update',data=groupedvalues)
plt.title('Türkiye Günlük İyileşme Sayıları');


In [ ]:
# Eksik veri sayıları ve veri setindeki oranları 
import matplotlib.pyplot as plt
import seaborn as sns
pd.concat([dataset.isnull().sum(), 100 * dataset.isnull().sum()/len(dataset)], 
              axis=1).rename(columns={0:'Missing Records', 1:'Percentage (%)'})

In [ ]:
dataset.isnull().sum().sum()

In [ ]:
dataset.isnull().sum()

In [ ]:
# Eksik Deger Tablosu Olusturalım

def eksik_deger_tablosu(dataset):
    eksik_deger=dataset.isnull().sum()
    eksik_deger_yuzde=100* dataset.isnull().sum()/len(dataset)
    eksik_deger_tablo= pd.concat([eksik_deger,eksik_deger_yuzde], axis=1)
    eksik_deger_tablo_son=eksik_deger_tablo.rename(
    columns = {0 : 'Eksik Değerler',1: '% Değeri'} )
    return eksik_deger_tablo_son

eksik_deger_tablosu(dataset)

In [ ]:
dataset.dropna()

In [ ]:
from sklearn.model_selection import train_test_split


X = dataset.iloc[:,2:5]
y = dataset['Recovered']

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=0)
X_train = mms.fit_transform(X_train) 
X_test= mms.fit_transform(X_test)
print("Dataframe boyutu: ",dataset.shape)
print("Eğitim verisi boyutu: ",X_train.shape, y_train.shape)
print("Test verisi boyutu: ",X_test.shape,y_test.shape)

In [ ]:
# type error için target typesı "Label Encoder" ile  multiclassa çevirdim.(Target=Y_train)
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y)
print(utils.multiclass.type_of_target(y))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(encoded))

lab_enc = preprocessing.LabelEncoder()
y_train = lab_enc.fit_transform(y_train)

In [ ]:
from sklearn    import metrics, svm
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import  linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Her bir modelin doğruluk değeri ,sınıflandırma raporu , karışıklık matrisi ve MSE(Ortalama Kare Hata Regresyon Oranı) değerlerini hesaplamak için import edildi.
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error

In [ ]:
# SVR(Support Vector Regressions)
print("SVR(Support Vector Regressions)")
clf = svm.SVR(gamma="auto")
# modelimizi eğitim verilerimiz ve buna karşılık gelen Y_train(target ) değerleri ile eğittik
clf.fit(X_train, y_train)
# test değerlerimize karşılık gelecek olan tahmin değerlerimizi oluşturduk
y_true2 , y_pred2 =y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred2)
plt.scatter(y_true2, y_pred2,c='black')
plt.scatter(y_true2, y_test,c='green')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
#SVR
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v1 = lab_enc.fit_transform(y_true2)
utils.multiclass.type_of_target(y_true2.astype('int'))
ypred2= lab_enc.fit_transform(y_pred2)
utils.multiclass.type_of_target(ypred2.astype('int'))
conf=confusion_matrix(encoded_v1, ypred2)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v1, ypred2))
print("\nClassification Report:\n",classification_report(encoded_v1, ypred2))
print("MSE:",mean_squared_error(encoded_v1, ypred2))

In [ ]:
# Lineer Regresyon
print("\nLineer Regresyon")
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
y_true1 , y_pred1 =y_test,lm.predict(X_test)
print("\nTahmin değerleri: ",y_pred1)
plt.scatter(y_true1, y_pred1,c='orange')
plt.scatter(y_true1, y_test,c='green')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
#Lineer Regresyon
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v = lab_enc.fit_transform(y_true1)
utils.multiclass.type_of_target(y_true1.astype('int'))
ypred1= lab_enc.fit_transform(y_pred1)
utils.multiclass.type_of_target(ypred1.astype('int'))
conf=confusion_matrix(encoded_v, ypred1)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v, ypred1))
print("\nClassification Report:\n",classification_report(encoded_v, ypred1))
print("MSE:",mean_squared_error(encoded_v, ypred1))

In [ ]:
# GaussianNB
print("GaussianNB")
clf = GaussianNB()
clf.fit(X_train, y_train)
y_true3 , y_pred3=y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred3)
plt.scatter(y_true3, y_pred3,c='grey')
plt.scatter(y_true3, y_test,c='green')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
# GaussianNB
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v2 = lab_enc.fit_transform(y_true3)
utils.multiclass.type_of_target(y_true3.astype('int'))
ypred3= lab_enc.fit_transform(y_pred3)
utils.multiclass.type_of_target(ypred3.astype('int'))
conf=confusion_matrix(encoded_v2, ypred3)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v2, ypred3))
print("\nClassification Report:\n",classification_report(encoded_v2, ypred3))
print("MSE:",mean_squared_error(encoded_v2, ypred3))

In [ ]:
# Decision Tree Classifier
print("Decision Tree Classifier")
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_true5 , y_pred5=y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred5)
plt.scatter(y_true5, y_pred5,c='brown')
plt.scatter(y_true5, y_test,c='green')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v4 = lab_enc.fit_transform(y_true5)
utils.multiclass.type_of_target(y_true5.astype('int'))
ypred5= lab_enc.fit_transform(y_pred5)
utils.multiclass.type_of_target(ypred5.astype('int'))
conf=confusion_matrix(encoded_v4, ypred5)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v4, ypred5))
print("\nClassification Report:\n",classification_report(encoded_v4, ypred5))
print("MSE:",mean_squared_error(encoded_v4, ypred5))

In [ ]:
# KNeighborsClassifier
print("KNeighbors Classifier")
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_true7 , y_pred7=y_test,clf.predict(X_test)
print("\nTahmin değerleri: ",y_pred7)
plt.scatter(y_true7, y_pred7,c='blue')
plt.scatter(y_true7, y_test,c='green')
plt.xlabel("True Values")
plt.ylabel("Predictions")

In [ ]:
#predictions multiclass olduğundan y_validation da multiclassa dönüştürüldü
encoded_v6 = lab_enc.fit_transform(y_true7)
utils.multiclass.type_of_target(y_true7.astype('int'))
ypred7= lab_enc.fit_transform(y_pred7)
utils.multiclass.type_of_target(ypred7.astype('int'))
conf=confusion_matrix(encoded_v6, ypred7)
print("\nConfusion matrix :\n",conf)
print("Accuracy score(Doğruluk değeri):\n",accuracy_score(encoded_v6, ypred7))
print("\nClassification Report:\n",classification_report(encoded_v6, ypred7))
print("MSE:",mean_squared_error(encoded_v6, ypred7))